# Making trisyllables from a csv file and .wavs of individual syllables
trisyllables are in snake case, with prominence and weight denoted as follows: 
NB: this works because python's string default is case sensitive.

       L, unstressed: cv
       L, prominent: cV
       H, unstressed: cvv
       H, prominent: cAA

for example:  

       QI: (LLL) 
              target pattern: learning 
              left edge: bA_ba_ba 
              right edge: ba_ba_bA

decision pairs: target + two incorrect options of possible positions of prominence: 

       dóloze, dolóze, dolozé
       * dO_lo_ze, do_lO_ze, dolozE
       each edge group has three: correct, incorrect1, incorrect2
              
       QS: 
       HLL: bAA_ba_ba 
       LHL: ba_bAA_ba
       H'HL: baa_bAA_ba

In [1]:
import pandas as pd

stim_input = "/Users/sarah/Git/stress_learn/syllables/"
trisyllables_df = pd.read_csv(stim_input + "stim.csv")
trisyllables_df.head()

,nonce,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,do_lo_ze,NaN,NaN,NaN
1,ba_da_go,NaN,NaN,NaN
2,ba_la_bo,NaN,NaN,NaN
3,lo_ge_za,NaN,NaN,NaN
4,le_ba_la,NaN,NaN,NaN


In [2]:

left = []
mid = []
right = []
#store the word in this row
for item in trisyllables_df.nonce: 
    nonce = item
    l_index = 1
    m = 4
    r_index = len(nonce)
    tmpLeft = nonce[0] + nonce[l_index].upper() + nonce[2:r_index]
    tmpMid = nonce[0:(m)] + nonce[m].upper() + nonce[(m + 1):r_index]
    tmpRight = nonce[0:r_index-1] + nonce[r_index-1].upper()

    left.append(tmpLeft)
    mid.append(tmpMid)
    right.append(tmpRight)
#print(left, mid, right)
trisyllables_df["left"] = left
trisyllables_df["mid"] = mid
trisyllables_df["right"] = right
trisyllables_df.head()


,nonce,Unnamed: 1,Unnamed: 2,Unnamed: 3,left,mid,right
0,do_lo_ze,NaN,NaN,NaN,dO_lo_ze,do_lO_ze,do_lo_zE
1,ba_da_go,NaN,NaN,NaN,bA_da_go,ba_dA_go,ba_da_gO
2,ba_la_bo,NaN,NaN,NaN,bA_la_bo,ba_lA_bo,ba_la_bO
3,lo_ge_za,NaN,NaN,NaN,lO_ge_za,lo_gE_za,lo_ge_zA
4,le_ba_la,NaN,NaN,NaN,lE_ba_la,le_bA_la,le_ba_lA


In [3]:
qi_conditions_df = trisyllables_df[["left","mid","right"]]
print(qi_conditions_df)

# print("AFTER CONVERSION\n", after, "\n")

        left       mid     right
0   dO_lo_ze  do_lO_ze  do_lo_zE
1   bA_da_go  ba_dA_go  ba_da_gO
2   bA_la_bo  ba_lA_bo  ba_la_bO
3   lO_ge_za  lo_gE_za  lo_ge_zA
4   lE_ba_la  le_bA_la  le_ba_lA
5   dA_lo_ze  da_lO_ze  da_lo_zE
6   zA_bo_lo  za_bO_lo  za_bo_lO
7   zA_be_le  za_bE_le  za_be_lE
8   dA_ba_de  da_bA_de  da_ba_dE
9   lO_ze_de  lo_zE_de  lo_ze_dE
10  gE_lo_de  ge_lO_de  ge_lo_dE
11  bO_le_lo  bo_lE_lo  bo_le_lO
12  lO_ba_be  lo_bA_be  lo_ba_bE
13  bE_lo_ge  be_lO_ge  be_lo_gE
14  gE_le_ba  ge_lE_ba  ge_le_bA
15  gO_de_za  go_dE_za  go_de_zA
16  bA_la_ge  ba_lA_ge  ba_la_gE
17  zE_bo_be  ze_bO_be  ze_bo_bE
18  zO_go_bo  zo_gO_bo  zo_go_bO
19  bE_ge_da  be_gE_da  be_ge_dA
20  dE_ge_ze  de_gE_ze  de_ge_zE
21  dO_la_be  do_lA_be  do_la_bE
22  lO_ge_ba  lo_gE_ba  lo_ge_bA
23  gA_le_bo  ga_lE_bo  ga_le_bO


In [4]:
import parselmouth
#set the directory for the .wav files of individual syllables
syll_root = "/Users/sarah/Git/stress_learn/syllables/"
#set the directory for new .wav files to be saved after concatenation
output_words = "/Users/sarah/Git/stress_learn/words/"


def affixer(trisyllable,output):
       #split string into individual syllables

    triplet = (trisyllable.split('_'))
    #print(triplet)
    init = triplet[0]
    penult = triplet[1]
    ultima = triplet[2]
    #print(init, penult, ultima)
    #create sound object of each syllable
    s1 = syll_root+"short_soft/" + init + ".wav"
    s2 = syll_root+ "short_soft/" + penult + ".wav"
    s3 = syll_root+ "short_stress/" + ultima + ".wav"
    syll1 = parselmouth.Sound(s1)
    syll2 = parselmouth.Sound(s2)
    syll3 = parselmouth.Sound(s3)
    #create list of sound objects
    tri_syll = [syll1,syll2,syll3]
    #tmpword = parselmouth.Sound.concatenate(syll1,syll2,0.0)
    word = parselmouth.Sound.concatenate(tri_syll,0.0)
    word.save((output + trisyllable+ ".wav"),parselmouth.SoundFileFormat.WAV)

for item in qi_conditions_df["right"]:
    affixer(item, output_words)